In [20]:
import os, code, sys, time
import numpy as np
import tensorflow as tf
import chainer
import chainer.functions as F
from chainer import cuda
from random import randrange
from numpy import linalg as LA
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import NearestNeighbors
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
import utils

In [21]:
'''
Session, network settings
'''
params_seed = 98765
data_seed   = 12345
def seed_rng(s=data_seed):
    np.random.seed(s)
    tf.set_random_seed(s)
    print('seeded by {}'.format(s))

use_gpu = True

In [ ]:
'''
Dataset parameters
'''
num_particles = 16 # defaults 16**3
zX = 0.6
zY = 0.0
X_input, X_truth = load_data(num_particles, zX, zY, normalize_data=True)
print('Using redshifts z{}, z{}, with {} particles'.format(zX,zY,num_particles**3))

In [22]:
'''
Model parameter initializations
'''
def init_weight(k_in, k_out):
    """ Initialize weights for fully connected layer
    weight drawn from he-normal distribution
    Args:
        k_in  (int): input channels
        k_out (int): output channels
    Returns: tf.Variable holding weight of shape (k_in, k_out)
    """
    henorm_std = np.sqrt(2.0 / k_in)
    weight = tf.random_normal((k_in, k_out), stddev=henorm_std)
    return tf.Variable(weight)

def init_graph_weights(k_in, k_out):
    """ initialize weights for graph layer
    Two weights:
    Wx : weight for external/hidden input
    Wg : weight for graph input
    """
    Wx = init_weight(k_in, k_out)
    Wg = init_weight(k_in, k_out)
    return Wx, Wg

def init_bias(k_in, k_out):
    """ initalize bias param
    Bias initialized to be near zero
    Returns: tf.Variable of shape (k_out,) for bias
    """
    
    bias = np.ones(k_out).astype(np.float32) * 1e-6
    return tf.Variable(bias)

def init_params(channels, graph_weights=True, use_bias=False):
    """ initializes all network hyperparameters
    Creates a dict with weights and biases associated with each
    hidden layer
    Args:
        channels (list): list of channel sizes
        graph_weights: if true, initializes weights for graph model
        use_bias: if true, bias params initialized, else None
    Returns: params dict containing weight and biases
    """
    weight_init_fun = init_graph_weights if graph_weights else init_weight
    kdims = [(channels[i], channels[i+1]) for i in range(len(channels) - 1)]    
    weights = []
    biases  = [] if use_bias else None
    for ktup in kdims:
        weights.append(weight_init_fun(*ktup))     
        if use_bias: biases.append(init_bias(*ktup))
    params = {'Weights': weights, 'Biases': biases}
    return params

In [25]:
'''
Model
'''
channels = [6, 8, 16, 32, 16, 8, 3, 8, 16, 32, 16, 8, 3]
learning_rate = 0.01
batch_size = 8
num_iters = 1000
seed_rng(params_seed)
graph_params = init_params(channels)

seeded by 98765


In [26]:
graph_params
'''
NOTE:
previous tf.Variables instantiaed though init_params are not destroyed
upon new calls to graph_params. You can see this by the tf.Variables'
default variable names. They just keep getting higher, instead of staying the same.
Maybe make a WEIGHT_LABEL and BIAS_LABEL global, init weights with 
variable name so that there are no unused Variables in graph
'''

{'Biases': None,
 'Weights': [(<tf.Variable 'Variable_72:0' shape=(6, 8) dtype=float32_ref>,
   <tf.Variable 'Variable_73:0' shape=(6, 8) dtype=float32_ref>),
  (<tf.Variable 'Variable_74:0' shape=(8, 16) dtype=float32_ref>,
   <tf.Variable 'Variable_75:0' shape=(8, 16) dtype=float32_ref>),
  (<tf.Variable 'Variable_76:0' shape=(16, 32) dtype=float32_ref>,
   <tf.Variable 'Variable_77:0' shape=(16, 32) dtype=float32_ref>),
  (<tf.Variable 'Variable_78:0' shape=(32, 16) dtype=float32_ref>,
   <tf.Variable 'Variable_79:0' shape=(32, 16) dtype=float32_ref>),
  (<tf.Variable 'Variable_80:0' shape=(16, 8) dtype=float32_ref>,
   <tf.Variable 'Variable_81:0' shape=(16, 8) dtype=float32_ref>),
  (<tf.Variable 'Variable_82:0' shape=(8, 3) dtype=float32_ref>,
   <tf.Variable 'Variable_83:0' shape=(8, 3) dtype=float32_ref>),
  (<tf.Variable 'Variable_84:0' shape=(3, 8) dtype=float32_ref>,
   <tf.Variable 'Variable_85:0' shape=(3, 8) dtype=float32_ref>),
  (<tf.Variable 'Variable_86:0' shape=(8, 1

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [18]:
graph_params['Weights'][3]

(<tf.Variable 'Variable_30:0' shape=(32, 16) dtype=float32_ref>,
 <tf.Variable 'Variable_31:0' shape=(32, 16) dtype=float32_ref>)

In [4]:
'''
Pre-processing: nearest-neighbors and sparse adjacency
'''

'''
SPARSE ADJACENCY MATRIX 
• scikit learn gives a CRS sparse adjacency for example, sparse TF takes COO. Use this for sparse X dense matmul in TF.
• also return adjacency lists and convert alist into index list to be used for generic normalizations (avg, max, etc)

tf has a great collection of sparse ops. Has a sparse/dense matmul, which may suit our needs. 
See which is faster/works:
tf.sparse_tensor_dense_matmul
tf.sparse_reduce
some other combination of csr.indptr and reduceat functions (not sure tf.reduce_mean can take reduction_along indices)
'''

# returns adjacency lists based on NN in coordinate space
def adjacency_list(X_in,k):
    shape_in = X_in.shape
    X_out = np.zeros([shape_in[0],shape_in[1],k],dtype=np.int32)
    for b in range(shape_in[0]):
        X_out[b] = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([shape_in[1],k])
    return X_out

def get_adjacency_list(X_in,k):
    """ search for k nneighbors, and return offsetted indices in adjacency list
    
    Args:
        X_in: input data of shape (mb_size, N, 6)
        k: number of nearest neighbors
    """
    mb_size, N, D = X_in.shape
    X_out = np.zeros([mb_size, N, k],dtype=np.int32)
    for b in range(mb_size):
        # this returns indices of the nn
        graph_idx = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([N,k]) + (N * b)
        X_out[b] = graph_idx
    return X_out

# adjacency list to proper index list for get_item
def alist_to_indexlist(alist):
    """ tiles batch indices to adjacency list for tf.gather
    """
    b, n, k = alist.shape
    #b = alist.shape[0] # batch size
    #n = alist.shape[1] # set size
    #k = alist.shape[2] # number of nn
    id1 = np.reshape(np.arange(b),[b,1])
    id1 = np.tile(id1,n*k).flatten()
    out = np.stack([id1,alist.flatten()],axis=1)
    return out